In [1]:
import os
import glob
import shutil

Remove unneeded image

In [ ]:
def move_rgb_files(parent_folder):
    
    for subfolder in os.listdir(parent_folder):
        subfolder_path = os.path.join(parent_folder, subfolder)
        if os.path.isdir(subfolder_path):
            color_folder = os.path.join(parent_folder, "Colored")

            if not os.path.exists(color_folder):
                os.makedirs(color_folder)


            for filename in os.listdir(subfolder_path):
                if 'RGB' in filename:
                    source_file = os.path.join(subfolder_path, filename)
                    dest_file = os.path.join(color_folder, filename)
                    
                    shutil.move(source_file, dest_file)
                    # print(f"moving: {filename} → {color_folder}")

    print("completed")

In [ ]:
imageFolder = "E:\\NYU\\pencreatic cancer AI\\0526\\NewData"
move_rgb_files(imageFolder)

To see if the photo is the same as previous

In [ ]:
import cv2
import numpy as np

def dyeing_one_color(photo_address):
    img = cv2.imread(photo_address)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lower_gray = 10
    upper_gray = 50
    mask = cv2.inRange(gray, lower_gray, upper_gray)

    # colored_img = img.copy()
    # colored_img[mask == 255] = [0, 0, 255]
    
    # cv2.imwrite('E:\\NYU\\pencreatic cancer AI\\others\\colored_output.png', colored_img)
    # cv2.imshow('Result', colored_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


In [ ]:
img1 = dyeing_one_color("E:\\NYU\\pencreatic cancer AI\\0526\\NewData\\Images 0\\DiD-MSLN_NYU360_5 percent 20ms_T001_XY1.tif")


Extract features from one image

In [69]:
import matplotlib.pyplot as plt

def extract_features(photo_address):
    gray_img = cv2.imread(photo_address, cv2.IMREAD_GRAYSCALE)
    _, binary_img = cv2.threshold(gray_img, 3, 255, cv2.THRESH_BINARY)
    # plt.imshow(binary_img, cmap='gray')
    # plt.title('Binary Image')
    # plt.axis('off')
    # plt.show()
    
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    one_img_trait = []
    
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        circularity = (4 * np.pi * area) / (perimeter ** 2 + 1e-6)
        
        (x, y), radius = cv2.minEnclosingCircle(contour)
        circle_area = np.pi * (radius ** 2)
        
        # brightness
        contour_mask = np.zeros(gray_img.shape, dtype=np.uint8)
        cv2.drawContours(contour_mask, [contour], -1, 255, -1)
        mean_val = cv2.mean(gray_img, mask=contour_mask)[0]
        
        result = [area, perimeter, circularity, radius, circle_area, mean_val]
        one_img_trait.append(result)
    
    results_array = np.array(one_img_trait)

    return (results_array)    

In [33]:
test_photo_add = "E:\\NYU\\pencreatic cancer AI\\0526\\NewData\\Images 0\\DiD-MSLN_NYU360_5 percent 20ms_T001_XY1.tif"

In [70]:
arr = extract_features(test_photo_add)
print(len(arr))

246


In [71]:
def load_sequences(root_dir):
    """
    Loads time-series feature sequences and labels
    Expects: /NewData
    Returns: X_seq, y_labels
    """
    X_seq = []
    y_labels = []


    for class_name, label in [('Images 0', 0), ('Images 0.5', 0.5), ('Images 1', 1)]:
        class_dir = os.path.join(root_dir, class_name)
        image_files = (os.listdir(class_dir))
        for each_img_name in image_files:
            each_img = os.path.join(class_dir, each_img_name)

            seq = extract_features(each_img)

            if len(seq) > 0:
                X_seq.append(seq)
                y_labels.append(label)

    return X_seq, y_labels

In [72]:
root_dir = "E:\\NYU\\pencreatic cancer AI\\0526\\NewData"
X_train_seq, y_train_labels = load_sequences(root_dir)
print(len(X_train_seq))

156


In [ ]:
print(len(y_train_labels))

In [ ]:
X_train_seq = np.array(X_train_seq)
y_train_labels = np.array(y_train_labels)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (156,) + inhomogeneous part.

In [68]:
for each in (X_train_seq):
    print(len(each))

246
72
109
187
39
76
85
238
71
104
199
55
83
87
255
80
109
184
54
75
92
265
75
111
191
49
71
91
240
83
105
184
51
76
99
238
81
111
181
51
73
88
240
76
125
161
52
79
87
245
80
111
26
99
105
3
155
24
91
96
3
146
22
91
108
3
156
21
92
103
3
152
20
87
110
3
153
20
86
102
3
147
22
85
105
3
151
29
82
102
3
159
26
83
101
3
149
25
86
105
3
151
28
83
4
2
3
1
13
4
3
5
2
4
3
12
4
3
4
3
4
1
12
4
3
3
3
4
1
16
4
3
5
2
5
1
12
4
2
4
2
6
1
10
5
3
4
2
6
1
19
5
2
4
3
5


In [74]:
# making the input with same length

def padding(list_of_arrays):
    max_rows = max(arr.shape[0] for arr in list_of_arrays)
    max_cols = max(arr.shape[1] for arr in list_of_arrays)
    def pad_array(array, target_rows, target_cols):
        padded = np.zeros((target_rows, target_cols))
        padded[:array.shape[0], :array.shape[1]] = array
        return padded
    
    padded_arrays = [pad_array(arr, max_rows, max_cols) for arr in list_of_arrays]
    
    padded_batch = np.stack(padded_arrays)
    print(f"batch shape: {padded_batch.shape}")

In [75]:
padding(X_train_seq)

batch shape: (156, 265, 6)


In [76]:
y_train_labels = np.array(y_train_labels)